## Importing BERT model

In [1]:
from transformers import BertTokenizer, BertModel

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3090


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [4]:
#checking tokenizer
tokenizer

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
}

## Importing CoLA dataset

### grammatically correct or not

In [5]:
import wget
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('/scratch/paneah/cola_public_1.1.zip'):
    wget.download(url, '/scratch/paneah/cola_public_1.1.zip')

# Unzip the dataset (if we haven't already)
if not os.path.exists('/scratch/paneah/cola_public'):
    !unzip /scratch/paneah/cola_public_1.1.zip -d /scratch/paneah/

In [6]:
!nvidia-smi

Thu Dec 14 22:58:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:81:00.0 Off |                  N/A |
| 37%   28C    P8               23W / 350W|      3MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
import pandas as pd

#loading dataset with pandas
df = pd.read_csv("/scratch/paneah/cola_public/raw/in_domain_train.tsv", \
                 delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
#printing number of sentences
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
#display random 10 rows 
df.sample(10)

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
5071,ks08,0,*,I believe that the world is round strongly.
2294,l-93,0,*,The witch turned him from a prince.
979,bc01,1,NaN,John bought the picture of himself that Bill saw.
8366,ad03,1,NaN,Aphrodite may quickly free the animals.
5857,c_13,1,NaN,The Brazilians pumped the oil across the river.
5367,b_73,1,NaN,Susan doesn't eat her vegetables enough.
4560,ks08,0,*,It has rain every day for the last week.
6403,d_98,1,NaN,Every woman standing under that tree is Mary's...
4356,ks08,1,NaN,There are believed to be sheep in the park.
6299,c_13,1,NaN,Frank will eat an apple and Morgan will too.


In [8]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Token IDs: [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


In [9]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/paneah/anaconda3/envs/LIS8040/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  Our friends won't buy this analysis, let alone the next one we propose.
Token IDs: tensor([  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
         2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [10]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

7,695 training samples
  856 validation samples


In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [12]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [13]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [14]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/home/paneah/anaconda3/envs/LIS8040/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [27]:
from tqdm import tqdm

seed_val = 42

torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch_i in tqdm(range(0, epochs), desc = "epoch", mininterval=0.01):
    total_train_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader), desc = "step", mininterval=0.01):

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
    
        


epoch:   0%|                                                                                      | 0/4 [00:00<?, ?it/s]

SequenceClassifierOutput(loss=tensor(0.8277, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.6166,  0.1010],
        [ 0.7037,  0.0917],
        [ 0.5382,  0.2110],
        [ 0.3998,  0.0816],
        [ 0.7558,  0.1214],
        [ 0.5495,  0.0447],
        [ 0.5952,  0.0849],
        [ 0.6200, -0.1042],
        [ 0.8252,  0.0635],
        [ 0.5924,  0.1349],
        [ 0.7005,  0.0552],
        [ 0.9809,  0.1697],
        [ 0.7981,  0.1985],
        [ 0.6155,  0.1968],
        [ 0.9245,  0.2495],
        [ 1.0990,  0.1884],
        [ 0.5896,  0.1726],
        [ 0.7901,  0.0817],
        [ 0.5910,  0.1578],
        [ 0.7196,  0.1819],
        [ 0.7065, -0.0479],
        [ 0.8457, -0.1434],
        [ 0.9219,  0.1112],
        [ 1.0231,  0.0543],
        [ 0.1673,  0.0660],
        [ 0.8025,  0.3730],
        [ 0.5688, -0.0068],
        [ 0.8737,  0.4722],
        [ 0.6920,  0.3135],
        [ 0.5603,  0.0166],
        [ 0.7782, -0.0880],
        [ 0.9404,  0.3009]], devic

SequenceClassifierOutput(loss=tensor(0.8913, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.6741,  0.1703],
        [ 0.9368,  0.3134],
        [ 0.8631,  0.0445],
        [ 0.9039,  0.1512],
        [ 1.0093,  0.0530],
        [ 0.5826,  0.2729],
        [ 0.8901,  0.1489],
        [ 0.4644, -0.0527],
        [ 0.8479,  0.3709],
        [ 0.5641, -0.2604],
        [ 0.7914,  0.2472],
        [ 0.7523,  0.2347],
        [ 0.6239, -0.0688],
        [ 0.7732,  0.0906],
        [ 0.6484, -0.0154],
        [ 1.0744,  0.0494],
        [ 0.8737,  0.2988],
        [ 0.7966,  0.0043],
        [ 0.3217, -0.2110],
        [ 0.7336,  0.0112],
        [ 1.1123,  0.0492],
        [ 0.9147,  0.0979],
        [ 0.7654, -0.1320],
        [ 0.5138, -0.0419],
        [ 0.7282, -0.0145],
        [ 0.6101,  0.1462],
        [ 0.6644, -0.1601],
        [ 0.8492,  0.0910],
        [ 0.5035, -0.0565],
        [ 0.7657, -0.1354],
        [ 0.6726, -0.0154],
        [ 0.6577,  0.0623]], devic

SequenceClassifierOutput(loss=tensor(0.8718, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.8013,  0.0165],
        [ 0.7721,  0.1568],
        [ 0.5262,  0.1656],
        [ 0.6242,  0.1874],
        [ 0.6123,  0.1490],
        [ 0.6285, -0.0356],
        [ 0.7219, -0.1199],
        [ 0.4162,  0.0638],
        [ 0.7555, -0.1570],
        [ 0.7468,  0.1609],
        [ 0.7483,  0.1573],
        [ 0.8780,  0.0436],
        [ 0.5386,  0.1808],
        [ 0.8515, -0.0087],
        [ 0.9322,  0.0745],
        [ 0.8046,  0.1786],
        [ 0.4342,  0.3201],
        [ 0.5046,  0.1219],
        [ 0.6050, -0.0177],
        [ 0.5709,  0.0630],
        [ 0.6432, -0.1031],
        [ 0.6741,  0.3958],
        [ 0.7630,  0.1847],
        [ 0.6751,  0.2565],
        [ 0.6180,  0.1092],
        [ 0.8566,  0.0625],
        [ 0.8524,  0.1339],
        [ 0.5476, -0.0033],
        [ 0.6616,  0.1793],
        [ 0.7525,  0.0500],
        [ 0.6805,  0.2232],
        [ 0.7609, -0.0425]], devic

SequenceClassifierOutput(loss=tensor(0.9334, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 1.1114, -0.1638],
        [ 0.6663, -0.0262],
        [ 0.8716, -0.1542],
        [ 0.8704,  0.4767],
        [ 0.7988,  0.0932],
        [ 0.5793,  0.2470],
        [ 0.5803,  0.1878],
        [ 0.9828,  0.0132],
        [ 0.4216,  0.1455],
        [ 0.5369,  0.2185],
        [ 0.9622,  0.3689],
        [ 0.7027,  0.0021],
        [ 0.8307,  0.0089],
        [ 0.4864, -0.0713],
        [ 1.0567,  0.2167],
        [ 0.2881,  0.0990],
        [ 0.7652,  0.3147],
        [ 0.1572,  0.1062],
        [ 0.7299, -0.0083],
        [ 0.7482, -0.0043],
        [ 0.4070,  0.1844],
        [ 0.5493,  0.0803],
        [ 0.3532, -0.0496],
        [ 0.5104, -0.1920],
        [ 0.2599,  0.1624],
        [ 0.7489,  0.1132],
        [ 0.7353,  0.0075],
        [ 0.9183,  0.1722],
        [ 0.8054,  0.2692],
        [ 0.5702, -0.0021],
        [ 0.5001,  0.2045],
        [ 0.6884,  0.1129]], devic

...

SequenceClassifierOutput(loss=tensor(1.0055, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.6703, -0.1125],
        [ 0.6031,  0.2513],
        [ 0.6551, -0.0130],
        [ 0.9297,  0.2770],
        [ 0.9000,  0.1555],
        [ 0.6589,  0.1622],
        [ 0.6545, -0.1054],
        [ 0.5069,  0.2491],
        [ 1.1330, -0.0496],
        [ 0.4114,  0.1564],
        [ 0.6501, -0.0743],
        [ 0.9854,  0.2197],
        [ 1.0233, -0.0719],
        [ 0.6115,  0.1869],
        [ 0.5444, -0.0238],
        [ 0.9812,  0.0471],
        [ 0.7711,  0.0613],
        [ 0.7830,  0.2704],
        [ 0.8942,  0.1800],
        [ 0.7889, -0.2090],
        [ 0.7862,  0.3013],
        [ 1.0688,  0.0319],
        [ 0.6202,  0.1651],
        [ 0.8969,  0.0722],
        [ 0.9103,  0.0903],
        [ 0.3374,  0.1550],
        [ 0.5771,  0.0327],
        [ 1.0407,  0.2399],
        [ 0.6117,  0.0467],
        [ 0.6275,  0.0802],
        [ 0.5964,  0.0961],
        [ 0.4625,  0.2812]], devic

SequenceClassifierOutput(loss=tensor(0.8113, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.2742, -0.0861],
        [ 0.7952,  0.1754],
        [ 0.5790,  0.0347],
        [ 0.7233,  0.1138],
        [ 0.5932, -0.0757],
        [ 0.9297,  0.4483],
        [ 0.6412, -0.0017],
        [ 0.7459,  0.1648],
        [ 1.0211,  0.1403],
        [ 0.4018,  0.0673],
        [ 0.8412,  0.3081],
        [ 0.1640, -0.1971],
        [ 0.7211,  0.1126],
        [ 0.5360,  0.1680],
        [ 0.6924,  0.1794],
        [ 0.5352, -0.1557],
        [ 0.7021,  0.1991],
        [ 0.9737,  0.3409],
        [ 0.6079,  0.0364],
        [ 0.4994,  0.2987],
        [ 0.4055, -0.0366],
        [ 0.3359,  0.0957],
        [ 0.4537, -0.0596],
        [ 0.5312,  0.1395],
        [ 0.9242,  0.1025],
        [ 0.2828, -0.1101],
        [ 0.8126,  0.1567],
        [ 0.5846, -0.0074],
        [ 0.7382,  0.4529],
        [ 0.4919, -0.1134],
        [ 0.5502, -0.0040],
        [ 0.9292,  0.0609]], devic

step: 210it [00:12, 16.88it/s]
epoch:   0%|                                                                                      | 0/4 [00:12<?, ?it/s]


SequenceClassifierOutput(loss=tensor(0.9246, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.4728,  0.1264],
        [ 0.8969,  0.1241],
        [ 0.7220,  0.2474],
        [ 0.7090,  0.1989],
        [ 0.4975, -0.1529],
        [ 0.6892,  0.1214],
        [ 0.5973, -0.1205],
        [ 0.8064, -0.0767],
        [ 0.9888, -0.1634],
        [ 0.9957,  0.4381],
        [ 0.6598, -0.0743],
        [ 0.7961,  0.4322],
        [ 0.6839,  0.0813],
        [ 0.5933,  0.1730],
        [ 0.5757, -0.2757],
        [ 0.7534,  0.0568],
        [ 0.5466, -0.0593],
        [ 0.2677,  0.1505],
        [ 0.8103, -0.0018],
        [ 0.7269,  0.0943],
        [ 0.8505, -0.1025],
        [ 0.9394, -0.0565],
        [ 1.1651,  0.2563],
        [ 0.7906,  0.0263],
        [ 0.6901, -0.1205],
        [ 0.7536,  0.1464],
        [ 0.6808,  0.0594],
        [ 0.9388,  0.2185],
        [ 0.2205,  0.2450],
        [ 0.7671,  0.0276],
        [ 0.6813, -0.0056],
        [ 1.1233,  0.0213]], devic

KeyboardInterrupt: 